In [22]:
# import statements
import os
import sys
from liquidhandling import SoloSoft, SoftLinx
from liquidhandling import *  # replace with the plate types you used 
from liquidhandling import DeepBlock_96VWR_75870_792_sterile
import pandas


In [34]:

# VARIABLES YOU CAN CHANGE ---------------------------------------------------------------------------------------------------------------

source_plate_files = [ # format = (unique plate name, csv file full path) 
    ("plate1", "/Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/test_plate_files/plate_csv_1.csv"), 
    ("plate2", "/Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/test_plate_files/plate_csv_2.csv"), 
    ("plate3", "/Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/test_plate_files/plate_csv_3.csv"),
    ("plate4", "/Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/test_plate_files/plate_csv_4.csv"), 
    ("plate5", "/Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/test_plate_files/plate_csv_5.csv")
]

save_to_directory_path = "/Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/created_files"   # path to the folder where you want to save all created files
softLinx_basename = "cherry_picking_test"  # don't include the .slvp extension
asp_source_volume = 10

# NOTE: the destination folder must already exist, this program will not create the folder for you

# DON"T CHANGE ANYTHING BELOW THIS LINE --------------------------------------------------------------------------------------------------------------

# calculate the number of rounds needed
source_plates_per_round = 4
num_rounds = int(len(source_plate_files)/source_plates_per_round)
num_rounds = num_rounds + 1 if (len(source_plate_files)%source_plates_per_round) else num_rounds


possible_source_dec_locs = [3,4,5,6] # do not change these!

destination_dictionary = {
    1:'A1', 2:'A2', 3:'A3', 4:'A4', 5:'A5', 6:'A6', 7:'A7', 8:'A8', 9:'A9', 10:'A10', 11:'A11', 
    12:'B1', 13:'B2', 14:'B3', 15:'B4', 16:'B5', 17:'B6', 18:'B7', 19:'B8', 20:'B9', 21:'B10', 22:'B11', 
    23:'C1', 24:'C2', 25:'C3', 26:'C4', 27:'C5', 28:'C6', 29:'C7', 30:'C8', 31:'C9', 32:'C10', 33:'C11',
    34:'D1', 35:'D2', 36:'D3', 37:'D4', 38:'D5', 39:'D6', 40:'D7', 41:'D8', 42:'D9', 43:'D10', 44:'D11',
} 

# extract plate_id and source wells from source plate csvs
transfers = []
for source_plate in source_plate_files: 
    plate_id, path = source_plate
        
    # read contents of each csv
    with open(path, "r", encoding='utf-8-sig') as open_csv:  # using 'utf-8-sig' ignores the \ufeff (what is that?)
        contents = open_csv.readlines()
        contents = [x.strip() for x in contents] 
        transfers.append((plate_id, contents))
        
# keep track of destination well and destination plate_location

transfers_list_copy = transfers.copy()  # copy of source plate array that I can mess with
dispense_count = 1
num_dipsense_plates = 1
is_dispense_plate_7 = True # True if dispensing into plate at position 7, false if dispensing into plate at position 8 (will flip as new plates are needeed)

for i in range(num_rounds): 
    # initialize SoftLinx (one softLinx file per round)
    softLinx_round_filename = f"{softLinx_basename}_ROUND{i+1}.slvp"
    softLinx_round_path = os.path.join(save_to_directory_path, softLinx_round_filename)
    softLinx = SoftLinx(softLinx_round_filename, softLinx_round_path)
    
    print(f"\nROUND {i+1}: (SoftLinx file = {softLinx_round_path} ")
    if len(transfers_list_copy) >= source_plates_per_round: 
        transfers_in_round = transfers_list_copy[:source_plates_per_round]
        transfers_list_copy = transfers_list_copy[source_plates_per_round:]
    else: 
        transfers_in_round = transfers_list_copy
        transfers_list_copy = []
        
    # Program variables
    tip_type = "TipBox.50uL.Axygen-EV-50-R-S.tealbox"
    source_plate_type = "DeepBlock.96.VWR-75870-792.sterile"
    destination_plate_type = "DeepBlock.96.VWR-75870-792.sterile"
    transfer_volume = 20
    default_z_shift = 2
#     blowoff = 5
        
    # loop through each set of transfers 
    for j in range(len(transfers_in_round)):
        plate_id, wells = transfers_in_round[j]
        plate_deck_location = possible_source_dec_locs[j]
        print(f"\n\tDeck location {plate_deck_location} --> {plate_id}")
        
        soloSoft_file_sep = 0
        
        for k in range(len(wells)):  
            # make a separate SoloSoft file per 15 transfers
            if k%15 == 0: 
                soloSoft_file_sep += 1
                current_filename = os.path.join(save_to_directory_path, f"round{i+1}_{plate_id}_step{soloSoft_file_sep}.hso")  # set filename for this round's solosoft file 
                print(f"\t\tSoloSoft filename = {current_filename})")
            
                 # initialize SoloSoft deck and filenames
                soloSoft = SoloSoft(
                    filename=current_filename,
                    plateList=[
                        tip_type,
                        "Empty",
                        source_plate_type,
                        source_plate_type,
                        source_plate_type,
                        source_plate_type,
                        destination_plate_type,
                        destination_plate_type,
                    ],
                )
                
            wells[k] = wells[k].strip() # remove any extra white spaces 
            soloSoft.getTip(num_tips=1)  # get only one tip
            
            # aspirate from specified well in specified source plate
            #TODO: Deep well version of the plates won't work so I had to use the reservoir version... should be fine
            soloSoft.aspirate(
                position=f"Position{plate_deck_location}",
                aspirate_volumes=Reservoir_12col_Agilent_201256_100_BATSgroup().setCell(wells[k][0], int(wells[k][1:]), transfer_volume),
                aspirate_shift=[0, 0, default_z_shift],
            )
            
            if dispense_count == 45: 
                dispense_count = 1
                is_dispense_plate_7 = False if is_dispense_plate_7 else True
                num_dipsense_plates += 1

            # determine dispensing plate and well locations
            dispense_well = destination_dictionary[dispense_count]
            dispense_count += 1

            dipsense_plate_pos = "Position7" if is_dispense_plate_7 else "Position8"
            print(f"\t\t\tasp: {wells[k]} --> disp: {dipsense_plate_pos}-{dispense_well}")
            
            # dispense into determined plate/well 
            soloSoft.dispense(
                position=dipsense_plate_pos,
                dispense_volumes=Reservoir_12col_Agilent_201256_100_BATSgroup().setCell(dispense_well[0], int(dispense_well[1:]), transfer_volume), 
                dispense_shift=[0,0,default_z_shift],
            )
            
            # shuck tip twice becuase sometimes the small tip gets stuck 
            soloSoft.shuckTip()
            soloSoft.shuckTip()
            soloSoft.savePipeline()
        softLinx.soloSoftRun(current_filename)

    softLinx.saveProtocol()      
            
print("\nPROTOCOL INFORMATION: ---------------------------------------------")         
print(f"\tNumber of source plates: {len(source_plate_files)}")
print(f"\tNumber of destination plates = {num_dipsense_plates}")  
print(f"\tNumber of rounds: {num_rounds}") 



ROUND 1: (SoftLinx file = /Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/created_files/cherry_picking_test_ROUND1.slvp 

	Deck location 3 --> plate1
		SoloSoft filename = /Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/created_files/round1_plate1_step1.hso)
			asp: A1 --> disp: Position7-A1
			asp: A3 --> disp: Position7-A2
			asp: A6 --> disp: Position7-A3
			asp: B2 --> disp: Position7-A4
			asp: C5 --> disp: Position7-A5
			asp: C6 --> disp: Position7-A6
			asp: C7 --> disp: Position7-A7
			asp: D1 --> disp: Position7-A8
			asp: E3 --> disp: Position7-A9
			asp: E4 --> disp: Position7-A10
			asp: E7 --> disp: Position7-A11
			asp: E9 --> disp: Position7-B1
			asp: E10 --> disp: Position7-B2
			asp: E11 --> disp: Position7-B3
			asp: E12 --> disp: Position7-B4
		SoloSoft filename = /Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/created_files/round1_plate1_step2.hso)
			asp: F2 --> disp: Position7-B5
			asp: G1 --